# Spark NLP for Healthcare

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Medical Text Summarization

MedicalSummarizer annotator that uses a type of transformative model, the T5 model, to create a concise summary of medical text given in a clinical context. This annotator helps to quickly summarize complex medical information.

## 📍Benchmark Report

Our clinical summarizer models with only 250M parameters perform 30-35% better than non-clinical SOTA text summarizers with 500M parameters, in terms of Bleu and Rouge benchmarks. That is, we achieve 30% better with half of the parameters that other LLMs have. See the details below.


### 🔎Benchmark on Samsum Dataset

| model_name | model_size | rouge | bleu | bertscore_precision | bertscore_recall: | bertscore_f1 |
|--|--|--|--|--|--|--|
philschmid/flan-t5-base-samsum | 240M | 0.2734 | 0.1813 | 0.8938 | 0.9133 | 0.9034 | 
linydub/bart-large-samsum | 500M | 0.3060 | 0.2168 | 0.8961 | 0.9065 | 0.9013 |
philschmid/bart-large-cnn-samsum | 500M | 0.3794 | 0.1262 | 0.8599 | 0.9153 | 0.8867 | 
transformersbook/pegasus-samsum | 570M | 0.3049 | 0.1543 | 0.8942 | 0.9183 | 0.9061 | 
summarizer_generic_jsl | 240M | 0.2703 | 0.1932 | 0.8944 | 0.9161 | 0.9051 |


### 🔎Benchmark on MtSamples Summarization Dataset

| model_name | model_size | rouge | bleu | bertscore_precision | bertscore_recall: | bertscore_f1 |
|--|--|--|--|--|--|--|
philschmid/flan-t5-base-samsum | 250M | 0.1919 | 0.1124 | 0.8409 | 0.8964 | 0.8678 | 
linydub/bart-large-samsum | 500M | 0.1586 | 0.0732 | 0.8747 | 0.8184 | 0.8456 | 
philschmid/bart-large-cnn-samsum |  500M | 0.2170 | 0.1299 | 0.8846 | 0.8436 | 0.8636 |
transformersbook/pegasus-samsum | 500M | 0.1924 | 0.0965 | 0.8920 | 0.8149 | 0.8517 | 
summarizer_clinical_jsl | 250M | 0.4836 | 0.4188 | 0.9041 | 0.9374 | 0.9204 | 
summarizer_clinical_jsl_augmented | 250M | 0.5119 | 0.4545 | 0.9282 | 0.9526 | 0.9402 |


### 🔎Benchmark on MIMIC Summarization Dataset

| model_name | model_size | rouge | bleu | bertscore_precision | bertscore_recall: | bertscore_f1 |
|--|--|--|--|--|--|--|
philschmid/flan-t5-base-samsum | 250M | 0.1910 | 0.1037 | 0.8708 | 0.9056 | 0.8879 | 
linydub/bart-large-samsum | 500M | 0.1252 | 0.0382 | 0.8933 | 0.8440 | 0.8679 |
philschmid/bart-large-cnn-samsum | 500M | 0.1795 | 0.0889 | 0.9172 | 0.8978 | 0.9074 | 
transformersbook/pegasus-samsum | 570M | 0.1425 | 0.0582 | 0.9171 | 0.8682 | 0.8920 |
summarizer_clinical_jsl | 250M | 0.395 | 0.2962 | 0.895 | 0.9316 | 0.913 | 
summarizer_clinical_jsl_augmented | 250M | 0.3964 | 0.307 | 0.9109 | 0.9452 | 0.9227 |

📌To run this yourself, you will need to upload your license keys to the notebook. Just Run The Cell Below in order to do that. Also You can open the file explorer on the left side of the screen and upload `license_keys.json` to the folder that opens.
Otherwise, you can look at the example outputs at the bottom of the notebook.

In [0]:
from johnsnowlabs import nlp, medical, visual
import pandas as pd
spark

🚨 Your Spark-Healthcare is outdated, installed==4.4.2 but latest version==4.4.1
You can run nlp.install() to update Spark-Healthcare
Out[2]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.1.2 
 Master 
 spark://10.139.64.4:7077 
 AppName 
 Databricks Shell

In [0]:
from pyspark.sql import DataFrame
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pyspark.sql as SQL
from pyspark import keyword_only

## 🔎 MODELS

<div align="center">

| **Index** | **Summarizer Models**        |
|---------------|----------------------|
| 1        | [summarizer_clinical_jsl](https://nlp.johnsnowlabs.com/2023/03/25/summarizer_clinical_jsl.html)     |
| 2          | [summarizer_clinical_jsl_augmented](https://nlp.johnsnowlabs.com/2023/03/30/summarizer_clinical_jsl_augmented_en.html)       |
| 3      | [summarizer_biomedical_pubmed](https://nlp.johnsnowlabs.com/2023/04/03/summarizer_biomedical_pubmed_en.html)    |
| 4      | [summarizer_generic_jsl](https://nlp.johnsnowlabs.com/2023/03/30/summarizer_generic_jsl_en.html)    |
| 5    | [summarizer_clinical_questions](https://nlp.johnsnowlabs.com/2023/04/03/summarizer_clinical_questions_en.html) |
| 6    | [summarizer_radiology](https://nlp.johnsnowlabs.com/2023/04/23/summarizer_jsl_radiology_en.html) |
| 7    | [summarizer_clinical_guidelines_large](https://nlp.johnsnowlabs.com/2023/05/08/summarizer_clinical_guidelines_large_en.html) |
</div>

## 📃 summarizer_clinical_jsl

Summarize clinical notes, encounters, critical care notes, discharge notes, reports, etc.

In [0]:
text = """ Patient with hypertension, syncope, and spinal stenosis - for recheck.
 (Medical Transcription Sample Report)
 SUBJECTIVE:
 The patient is a 78-year-old female who returns for recheck. She has hypertension. She denies difficulty with chest pain, palpations, orthopnea, nocturnal dyspnea, or edema.
 PAST MEDICAL HISTORY / SURGERY / HOSPITALIZATIONS:
 Reviewed and unchanged from the dictation on 12/03/2003.
 MEDICATIONS:
 Atenolol 50 mg daily, Premarin 0.625 mg daily, calcium with vitamin D two to three pills daily, multivitamin daily, aspirin as needed, and TriViFlor 25 mg two pills daily. She also has Elocon cream 0.1% and Synalar cream 0.01% that she uses as needed for rash."""
 
 
data = spark.createDataFrame([[text]]).toDF("text")
data.show(truncate = 60)

+------------------------------------------------------------+
 text|
+------------------------------------------------------------+
 Patient with hypertension, syncope, and spinal stenosis ...|
+------------------------------------------------------------+

In [0]:
document_assembler = nlp.DocumentAssembler()\
            .setInputCol('text')\
            .setOutputCol('document')

summarizer = medical.Summarizer.pretrained("summarizer_clinical_jsl", "en", "clinical/models")\
            .setInputCols(['document'])\
            .setOutputCol('summary')\
            .setMaxTextLength(512)\
            .setMaxNewTokens(512)

pipeline = nlp.Pipeline(stages=[
            document_assembler,
            summarizer])

model = pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

result = model.transform(data)

result.show()

summarizer_clinical_jsl download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]
+--------------------+--------------------+--------------------+
 text| document| summary|
+--------------------+--------------------+--------------------+
 Patient with hyp...|[{document, 0, 68...|[{document, 0, 24...|
+--------------------+--------------------+--------------------+

In [0]:
result.select("summary.result").show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
result |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
[A 78-year-old female with hypertension, syncope, and spinal stenosis returns for recheck. She denies chest pain, palpations, orthopnea, nocturnal dyspnea, or edema. She is on multiple medications and has Elocon cream and Synalar cream for rash.]|
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+

### 📍 LightPipelines

In [0]:
text = """The patient is a pleasant 17-year-old gentleman who was playing basketball today in gym. Two hours prior to presentation, he started to fall and someone stepped on his ankle and kind of twisted his right ankle and he cannot bear weight on it now. It hurts to move or bear weight. No other injuries noted. He does not think he has had injuries to his ankle in the past.
SOCIAL HISTORY: He does not drink or smoke.
MEDICAL DECISION MAKING:
He had an x-ray of his ankle that showed a small ossicle versus avulsion fracture of the talonavicular joint on the lateral view. He has had no pain over the metatarsals themselves. This may be a fracture based upon his exam. He does want to have me to put him in a splint. He was given Motrin here. He will be discharged home to follow up with Dr. X from Orthopedics.
DISPOSITION: Crutches and splint were administered here. I gave him a prescription for Motrin and some Darvocet if he needs to length his sleep and if he has continued pain to follow up with Dr. X. Return if any worsening problems."""

light_model = nlp.LightPipeline(model)
light_result = light_model.annotate(text)
light_result

Out[7]: {'document': ['The patient is a pleasant 17-year-old gentleman who was playing basketball today in gym. Two hours prior to presentation, he started to fall and someone stepped on his ankle and kind of twisted his right ankle and he cannot bear weight on it now. It hurts to move or bear weight. No other injuries noted. He does not think he has had injuries to his ankle in the past.\nSOCIAL HISTORY: He does not drink or smoke.\nMEDICAL DECISION MAKING:\nHe had an x-ray of his ankle that showed a small ossicle versus avulsion fracture of the talonavicular joint on the lateral view. He has had no pain over the metatarsals themselves. This may be a fracture based upon his exam. He does want to have me to put him in a splint. He was given Motrin here. He will be discharged home to follow up with Dr. X from Orthopedics.\nDISPOSITION: Crutches and splint were administered here. I gave him a prescription for Motrin and some Darvocet if he needs to length his sleep and if he has continued pain to follow up with Dr. X. Return if any worsening problems.'],
 'summary': ['A 17-year-old man fell and twisted his right ankle, causing pain to move or bear weight. An x-ray showed a small ossicle or avulsion fracture of the talonavicular joint on the lateral view, which may be a fracture based upon his exam. He was given Motrin and discharged home with crutches and a prescription for Motrin and Darvocet. He was advised to follow up with his doctor if pain worsens and return if any worsening problems worsen.']}

In [0]:
light_result["summary"]

Out[8]: ['A 17-year-old man fell and twisted his right ankle, causing pain to move or bear weight. An x-ray showed a small ossicle or avulsion fracture of the talonavicular joint on the lateral view, which may be a fracture based upon his exam. He was given Motrin and discharged home with crutches and a prescription for Motrin and Darvocet. He was advised to follow up with his doctor if pain worsens and return if any worsening problems worsen.']

### 🚩 summaries from paragraphs in text

In [0]:
document_assembler = nlp.DocumentAssembler()\
            .setInputCol('text')\
            .setOutputCol('document')
 
sentenceDetector = nlp.SentenceDetectorDLModel\
            .pretrained("sentence_detector_dl_healthcare","en","clinical/models")\
            .setInputCols(["document"])\
            .setOutputCol("sentence")\
            .setCustomBounds(["\n"])\
            .setUseCustomBoundsOnly(True)

summarizer = medical.Summarizer\
            .pretrained("summarizer_clinical_jsl")\
            .setInputCols(['sentence'])\
            .setOutputCol('summary')\
            .setMaxTextLength(512)\
            .setMaxNewTokens(512)

pipeline = nlp.Pipeline(stages=[
            document_assembler,
            sentenceDetector,
            summarizer])

model = pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[ | ][OK!]
summarizer_clinical_jsl download started this may take some time.
[ | ][OK!]

In [0]:
text = """PRESENT ILLNESS: The patient is a 28-year-old, who is status post gastric bypass surgery nearly one year ago. He has lost about 200 pounds and was otherwise doing well until yesterday evening around 7:00-8:00 when he developed nausea and right upper quadrant pain, which apparently wrapped around toward his right side and back. He feels like he was on it but has not done so. He has overall malaise and a low-grade temperature of 100.3. He denies any prior similar or lesser symptoms. His last normal bowel movement was yesterday. He denies any outright chills or blood per rectum.

PHYSICAL EXAMINATION: His temperature is 100.3, blood pressure 129/59, respirations 16, heart rate 84. He is drowsy, but easily arousable and appropriate with conversation. He is oriented to person, place, and situation. He is normocephalic, atraumatic. His sclerae are anicteric. His mucous membranes are somewhat tacky. His neck is supple and symmetric. His respirations are unlabored and clear. He has a regular rate and rhythm. His abdomen is soft. He has diffuse right upper quadrant tenderness, worse focally, but no rebound or guarding. He otherwise has no organomegaly, masses, or abdominal hernias evident. His extremities are symmetrical with no edema. His posterior tibial pulses are palpable and symmetric. He is grossly nonfocal neurologically.

PLAN: He will be admitted and placed on IV antibiotics. We will get an ultrasound this morning. He will need his gallbladder out, probably with intraoperative cholangiogram. Hopefully, the stone will pass this way. Due to his anatomy, an ERCP would prove quite difficult if not impossible unless laparoscopic assisted. Dr. X will see him later this morning and discuss the plan further. The patient understands."""

light_model = nlp.LightPipeline(model)
light_result = light_model.annotate(text)

In [0]:
import textwrap

for i in range(len(light_result['sentence'])):
    document_text = textwrap.fill(light_result['sentence'][i], width=120)
    summary_text = textwrap.fill(light_result['summary'][i], width=120)
    
    print("➤ Document {}: \n{}".format(i+1, document_text))
    print("\n")
    print("➤ Summary {}: \n{}".format(i+1, summary_text))
    print("\n")

➤ Document 1: 
PRESENT ILLNESS: The patient is a 28-year-old, who is status post gastric bypass surgery nearly one year ago. He has
lost about 200 pounds and was otherwise doing well until yesterday evening around 7:00-8:00 when he developed nausea and
right upper quadrant pain, which apparently wrapped around toward his right side and back. He feels like he was on it
but has not done so. He has overall malaise and a low-grade temperature of 100.3. He denies any prior similar or lesser
symptoms. His last normal bowel movement was yesterday. He denies any outright chills or blood per rectum.


➤ Summary 1: 
A 28-year-old patient who had gastric bypass surgery nearly one year ago developed nausea and right upper quadrant pain,
which wrapped around his right side and back. He has malaise and a low-grade temperature of 100.3. He denies any
previous symptoms and has no other symptoms.


➤ Document 2: 
PHYSICAL EXAMINATION: His temperature is 100.3, blood pressure 129/59, respirations 16, heart rate 84. He is drowsy, but
easily arousable and appropriate with conversation. He is oriented to person, place, and situation. He is normocephalic,
atraumatic. His sclerae are anicteric. His mucous membranes are somewhat tacky. His neck is supple and symmetric. His
respirations are unlabored and clear. He has a regular rate and rhythm. His abdomen is soft. He has diffuse right upper
quadrant tenderness, worse focally, but no rebound or guarding. He otherwise has no organomegaly, masses, or abdominal
hernias evident. His extremities are symmetrical with no edema. His posterior tibial pulses are palpable and symmetric.
He is grossly nonfocal neurologically.


➤ Summary 2: 
The patient has a temperature of 100.3, blood pressure of 129/59, respirations of 16, and heart rate of 84. He is drowsy
but oriented to person, place, and situation. He has normalocephalic, atraumatic, and supple and symmetric neck. His
respiratory rhythm is unlabored and clear, and his abdomen is soft. He has diffuse right upper quadrant tenderness,
worse focally, but no rebound or guarding. He has no organomegaly, masses, or abdominal hernias evident. His extremities
are symmetrical with no edema, and his posterior tibial pulses are palpable and symmetric. He is grossly nonfocal
neurologically.


➤ Document 3: 
PLAN: He will be admitted and placed on IV antibiotics. We will get an ultrasound this morning. He will need his
gallbladder out, probably with intraoperative cholangiogram. Hopefully, the stone will pass this way. Due to his
anatomy, an ERCP would prove quite difficult if not impossible unless laparoscopic assisted. Dr. X will see him later
this morning and discuss the plan further. The patient understands.


➤ Summary 3: 
The patient will be admitted and placed on IV antibiotics. An ultrasound will be done to remove the gallbladder, which
will be done with intraoperative cholangiogram. The stone will pass through the gallbladder, and an ERCP is difficult
due to the patient's anatomy. The plan will be discussed with the patient later this morning.

### 🚩 setRefineSummary

**We've Made Significant Enhancements To Our Text Summarization Method, Which Now Utilizes A Map-Reduce Approach For Section-Wise Summarization.**

We are excited to announce the integration of new parameters into our `MedicalSummarizer` annotator, empowering users to overcome token limitations and attain heightened flexibility in their medical summarization endeavors. These advanced parameters significantly augment the annotator's functionality, enabling users to generate more accurate and comprehensive summaries of medical documents. By employing a map-reduce approach, the `MedicalSummarizer` efficiently condenses distinct text segments until the desired length is achieved.

The following parameters have been introduced:

- `setRefineSummary`: Set to True for refined summarization with increased computational cost.
- `setRefineSummaryTargetLength`: Define the target length of summarizations in tokens (delimited by whitespace). Effective only when setRefineSummary=True.
- `setRefineChunkSize`: Specify the desired size of refined chunks. Should correspond to the LLM context window size in tokens. Effective only when - `setRefineSummary=True`.
- `setRefineMaxAttempts`: Determine the number of attempts for re-summarizing chunks exceeding the setRefineSummaryTargetLength before discontinuing. Effective only when `setRefineSummary=True`.

These enhancements to the MedicalSummarizer annotator represent our ongoing commitment to providing state-of-the-art tools for healthcare professionals and researchers, facilitating more efficient and accurate medical text analysis.

In [0]:
document_assembler = nlp.DocumentAssembler()\
            .setInputCol('text')\
            .setOutputCol('document')

summarizer = medical.Summarizer.pretrained("summarizer_clinical_jsl", "en", "clinical/models")\
            .setInputCols(["document"])\
            .setOutputCol("summary")\
            .setMaxTextLength(512)\
            .setMaxNewTokens(512)\
            .setDoSample(True)\
            .setRefineSummary(True)\
            .setRefineSummaryTargetLength(100)\
            .setRefineMaxAttempts(3)\
            .setRefineChunkSize(512)\
            
pipeline = nlp.Pipeline(stages=[
            document_assembler,
            summarizer])

model = pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

summarizer_clinical_jsl download started this may take some time.
[ | ][OK!]

In [0]:
text = """The patient is a pleasant 17-year-old gentleman who was playing basketball today in gym. Two hours prior to presentation, he started to fall and someone stepped on his ankle and kind of twisted his right ankle and he cannot bear weight on it now. It hurts to move or bear weight. No other injuries noted. He does not think he has had injuries to his ankle in the past.
SOCIAL HISTORY: He does not drink or smoke.
MEDICAL DECISION MAKING:
He had an x-ray of his ankle that showed a small ossicle versus avulsion fracture of the talonavicular joint on the lateral view. He has had no pain over the metatarsals themselves. This may be a fracture based upon his exam. He does want to have me to put him in a splint. He was given Motrin here. He will be discharged home to follow up with Dr. X from Orthopedics.
DISPOSITION: Crutches and splint were administered here. I gave him a prescription for Motrin and some Darvocet if he needs to length his sleep and if he has continued pain to follow up with Dr. X. Return if any worsening problems."""

light_model = nlp.LightPipeline(model)

light_result = light_model.annotate(text)

In [0]:
light_result["summary"]

Out[14]: ['A 28-year-old patient who had gastric bypass surgery nearly one year ago developed nausea and right upper quadrant pain, which wrapped around his right side and back. He has malaise and a low-grade temperature of 100.3. He denies any previous symptoms and has no other symptoms.',
 'The patient has a temperature of 100.3, blood pressure of 129/59, respirations of 16, and heart rate of 84. He is drowsy but oriented to person, place, and situation. He has normalocephalic, atraumatic, and supple and symmetric neck. His respiratory rhythm is unlabored and clear, and his abdomen is soft. He has diffuse right upper quadrant tenderness, worse focally, but no rebound or guarding. He has no organomegaly, masses, or abdominal hernias evident. His extremities are symmetrical with no edema, and his posterior tibial pulses are palpable and symmetric. He is grossly nonfocal neurologically.',
 "The patient will be admitted and placed on IV antibiotics. An ultrasound will be done to remove the gallbladder, which will be done with intraoperative cholangiogram. The stone will pass through the gallbladder, and an ERCP is difficult due to the patient's anatomy. The plan will be discussed with the patient later this morning."]

In [0]:
import textwrap

for i in range(len(light_result['document'])):
    document_text = textwrap.fill(light_result['document'][i], width=120)
    summary_text = textwrap.fill(light_result['summary'][i], width=120)
    
    print("➤ Document {}: \n{}".format(i+1, document_text))
    print("\n")
    print("➤ Summary {}: \n{}".format(i+1, summary_text))
    print("\n")

➤ Document 1: 
The patient is a pleasant 17-year-old gentleman who was playing basketball today in gym. Two hours prior to
presentation, he started to fall and someone stepped on his ankle and kind of twisted his right ankle and he cannot bear
weight on it now. It hurts to move or bear weight. No other injuries noted. He does not think he has had injuries to his
ankle in the past. SOCIAL HISTORY: He does not drink or smoke. MEDICAL DECISION MAKING: He had an x-ray of his ankle
that showed a small ossicle versus avulsion fracture of the talonavicular joint on the lateral view. He has had no pain
over the metatarsals themselves. This may be a fracture based upon his exam. He does want to have me to put him in a
splint. He was given Motrin here. He will be discharged home to follow up with Dr. X from Orthopedics. DISPOSITION:
Crutches and splint were administered here. I gave him a prescription for Motrin and some Darvocet if he needs to length
his sleep and if he has continued pain to follow up with Dr. X. Return if any worsening problems.


➤ Summary 1: 
An 17-month patient undertook surgical treatment for ankle twisted injuries due to the occulic bone fragments, including
medication and pain medication for pain management. He also had a stress ulcer and blood level monitor during
presentation. He also was advised to avoid smoking or drinking, maintain blood-alkaloid and blood sugar control, and
monitor for any withdrawal symptoms. The plan for any other injuries is being pushed back into action. No more further
surgical options have taken.

## 📃 summarizer_clinical_jsl_augmented

In [0]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol('text')\
    .setOutputCol('document')
            
med_summarizer  = medical.Summarizer()\
    .pretrained("summarizer_clinical_jsl_augmented")\
    .setInputCols("document")\
    .setOutputCol("summary")\
    .setMaxNewTokens(100)\
    .setMaxTextLength(1024)
    
pipeline = nlp.Pipeline(stages=[document_assembler, 
                            med_summarizer])


model = pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))
light_model = nlp.LightPipeline(model)

summarizer_clinical_jsl_augmented download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]

In [0]:
text = """Residual disease after initial surgery for ovarian cancer is the strongest prognostic factor for survival. However, the extent of surgical resection required to achieve optimal cytoreduction is controversial. Our goal was to estimate the effect of aggressive surgical resection on ovarian cancer patient survival.\\n A retrospective cohort study of consecutive patients with International Federation of Gynecology and Obstetrics stage IIIC ovarian cancer undergoing primary surgery was conducted between January 1, 1994, and December 31, 1998. The main outcome measures were residual disease after cytoreduction, frequency of radical surgical resection, and 5-year disease-specific survival.\\n The study comprised 194 patients, including 144 with carcinomatosis. The mean patient age and follow-up time were 64.4 and 3.5 years, respectively. After surgery, 131 (67.5%) of the 194 patients had less than 1 cm of residual disease (definition of optimal cytoreduction). Considering all patients, residual disease was the only independent predictor of survival; the need to perform radical procedures to achieve optimal cytoreduction was not associated with a decrease in survival. For the subgroup of patients with carcinomatosis, residual disease and the performance of radical surgical procedures were the only independent predictors. Disease-specific survival was markedly improved for patients with carcinomatosis operated on by surgeons who most frequently used radical procedures compared with those least likely to use radical procedures (44% versus 17%, P < .001).\\n Overall, residual disease was the only independent predictor of survival. Minimizing residual disease through aggressive surgical resection was beneficial, especially in patients with carcinomatosis."""

light_result = light_model.annotate(text)
light_result

Out[20]: {'document': ['Residual disease after initial surgery for ovarian cancer is the strongest prognostic factor for survival. However, the extent of surgical resection required to achieve optimal cytoreduction is controversial. Our goal was to estimate the effect of aggressive surgical resection on ovarian cancer patient survival.\\n A retrospective cohort study of consecutive patients with International Federation of Gynecology and Obstetrics stage IIIC ovarian cancer undergoing primary surgery was conducted between January 1, 1994, and December 31, 1998. The main outcome measures were residual disease after cytoreduction, frequency of radical surgical resection, and 5-year disease-specific survival.\\n The study comprised 194 patients, including 144 with carcinomatosis. The mean patient age and follow-up time were 64.4 and 3.5 years, respectively. After surgery, 131 (67.5%) of the 194 patients had less than 1 cm of residual disease (definition of optimal cytoreduction). Considering all patients, residual disease was the only independent predictor of survival; the need to perform radical procedures to achieve optimal cytoreduction was not associated with a decrease in survival. For the subgroup of patients with carcinomatosis, residual disease and the performance of radical surgical procedures were the only independent predictors. Disease-specific survival was markedly improved for patients with carcinomatosis operated on by surgeons who most frequently used radical procedures compared with those least likely to use radical procedures (44% versus 17%, P < .001).\\n Overall, residual disease was the only independent predictor of survival. Minimizing residual disease through aggressive surgical resection was beneficial, especially in patients with carcinomatosis.'],
 'summary': ['The study of a patient with stage IIIC ovarian cancer undergoing primary surgery revealed residual disease after initial surgery, frequency of radical surgical resection, and 5-year disease-specific survival. The study showed that aggressive surgical resection was the only independent predictor of survival, with less than 1 cm of residual disease. Disease-specific survival was improved for patients with carcinomatosis operated on by surgeons who most frequently used radical procedures compared to those least likely to use']}

In [0]:
light_result["summary"]

Out[21]: ['The study of a patient with stage IIIC ovarian cancer undergoing primary surgery revealed residual disease after initial surgery, frequency of radical surgical resection, and 5-year disease-specific survival. The study showed that aggressive surgical resection was the only independent predictor of survival, with less than 1 cm of residual disease. Disease-specific survival was improved for patients with carcinomatosis operated on by surgeons who most frequently used radical procedures compared to those least likely to use']

## 📃 summarizer_biomedical_pubmed
This model is a modified version of Flan-T5 (LLM) based summarization model that is finetuned with biomedical datasets (Pubmed abstracts) by John Snow Labs.  It can generate summaries up to 512 tokens given an input text (max 1024 tokens).

In [0]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

summarizer = medical.Summarizer()\
    .pretrained("summarizer_biomedical_pubmed")\
    .setInputCols(["document"])\
    .setOutputCol("summary")\
    .setMaxTextLength(512)\
    .setMaxNewTokens(512)

pipeline = nlp.Pipeline(stages=[
    document_assembler,
    summarizer  
])

model = pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))
light_model = nlp.LightPipeline(model)

summarizer_biomedical_pubmed download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]

In [0]:
text = """Residual disease after initial surgery for ovarian cancer is the strongest prognostic factor for survival. However, the extent of surgical resection required to achieve optimal cytoreduction is controversial. Our goal was to estimate the effect of aggressive surgical resection on ovarian cancer patient survival.\\n A retrospective cohort study of consecutive patients with International Federation of Gynecology and Obstetrics stage IIIC ovarian cancer undergoing primary surgery was conducted between January 1, 1994, and December 31, 1998. The main outcome measures were residual disease after cytoreduction, frequency of radical surgical resection, and 5-year disease-specific survival.\\n The study comprised 194 patients, including 144 with carcinomatosis. The mean patient age and follow-up time were 64.4 and 3.5 years, respectively. After surgery, 131 (67.5%) of the 194 patients had less than 1 cm of residual disease (definition of optimal cytoreduction). Considering all patients, residual disease was the only independent predictor of survival; the need to perform radical procedures to achieve optimal cytoreduction was not associated with a decrease in survival. For the subgroup of patients with carcinomatosis, residual disease and the performance of radical surgical procedures were the only independent predictors. Disease-specific survival was markedly improved for patients with carcinomatosis operated on by surgeons who most frequently used radical procedures compared with those least likely to use radical procedures (44% versus 17%, P < .001).\\n Overall, residual disease was the only independent predictor of survival. Minimizing residual disease through aggressive surgical resection was beneficial, especially in patients with carcinomatosis."""

light_result = light_model.annotate(text)
light_result

Out[23]: {'document': ['Residual disease after initial surgery for ovarian cancer is the strongest prognostic factor for survival. However, the extent of surgical resection required to achieve optimal cytoreduction is controversial. Our goal was to estimate the effect of aggressive surgical resection on ovarian cancer patient survival.\\n A retrospective cohort study of consecutive patients with International Federation of Gynecology and Obstetrics stage IIIC ovarian cancer undergoing primary surgery was conducted between January 1, 1994, and December 31, 1998. The main outcome measures were residual disease after cytoreduction, frequency of radical surgical resection, and 5-year disease-specific survival.\\n The study comprised 194 patients, including 144 with carcinomatosis. The mean patient age and follow-up time were 64.4 and 3.5 years, respectively. After surgery, 131 (67.5%) of the 194 patients had less than 1 cm of residual disease (definition of optimal cytoreduction). Considering all patients, residual disease was the only independent predictor of survival; the need to perform radical procedures to achieve optimal cytoreduction was not associated with a decrease in survival. For the subgroup of patients with carcinomatosis, residual disease and the performance of radical surgical procedures were the only independent predictors. Disease-specific survival was markedly improved for patients with carcinomatosis operated on by surgeons who most frequently used radical procedures compared with those least likely to use radical procedures (44% versus 17%, P < .001).\\n Overall, residual disease was the only independent predictor of survival. Minimizing residual disease through aggressive surgical resection was beneficial, especially in patients with carcinomatosis.'],
 'summary': ['The results of this review suggest that aggressive ovarian cancer surgery is associated with a significant reduction in the risk of recurrence and a reduction in the number of radical versus conservative surgical resections. However, the results of this review are based on only one small trial. Further research is needed to determine the role of aggressive ovarian cancer surgery in women with stage IIIC ovarian cancer.']}

In [0]:
light_result["summary"]

Out[24]: ['The results of this review suggest that aggressive ovarian cancer surgery is associated with a significant reduction in the risk of recurrence and a reduction in the number of radical versus conservative surgical resections. However, the results of this review are based on only one small trial. Further research is needed to determine the role of aggressive ovarian cancer surgery in women with stage IIIC ovarian cancer.']

## 📃 summarizer_clinical_questions

This model is a modified version of Flan-T5 (LLM) based summarization model that is finetuned with medical questions exchanged in clinical mediums (clinic, email, call center etc.) by John Snow Labs.  It can generate summaries up to 512 tokens given an input text (max 1024 tokens).

In [0]:
summarizer = medical.Summarizer()\
    .pretrained("summarizer_clinical_questions")\
    .setInputCols(["document"])\
    .setOutputCol("summary")\
    .setMaxTextLength(512)\
    .setMaxNewTokens(512)

pipeline = nlp.Pipeline(stages=[
    document_assembler,
    summarizer  
])

model = pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))
light_model = nlp.LightPipeline(model)

summarizer_clinical_questions download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]

In [0]:
text = """ 
Hello,I'm 20 year old girl. I'm diagnosed with hyperthyroid 1 month ago. I was feeling weak, light headed,poor digestion, panic attacks, depression, left chest pain, increased heart rate, rapidly weight loss, from 4 months. Because of this, I stayed in the hospital and just discharged from hospital. I had many other blood tests, brain mri, ultrasound scan, endoscopy because of some dumb doctors bcs they were not able to diagnose actual problem. Finally I got an appointment with a homeopathy doctor finally he find that i was suffering from hyperthyroid and my TSH was 0.15 T3 and T4 is normal . Also i have b12 deficiency and vitamin D deficiency so I'm taking weekly supplement of vitamin D and 1000 mcg b12 daily. I'm taking homeopathy medicine for 40 days and took 2nd test after 30 days. My TSH is 0.5 now. I feel a little bit relief from weakness and depression but I'm facing with 2 new problem from last week that is breathtaking problem and very rapid heartrate. I just want to know if i should start allopathy medicine or homeopathy is okay? Bcs i heard that thyroid take time to start recover. So please let me know if both of medicines take same time. Because some of my friends advising me to start allopathy and never take a chance as i can develop some serious problems.Sorry for my poor english😐Thank you.
"""

light_result = light_model.annotate(text)
light_result


Out[26]: {'document': [" \nHello,I'm 20 year old girl. I'm diagnosed with hyperthyroid 1 month ago. I was feeling weak, light headed,poor digestion, panic attacks, depression, left chest pain, increased heart rate, rapidly weight loss, from 4 months. Because of this, I stayed in the hospital and just discharged from hospital. I had many other blood tests, brain mri, ultrasound scan, endoscopy because of some dumb doctors bcs they were not able to diagnose actual problem. Finally I got an appointment with a homeopathy doctor finally he find that i was suffering from hyperthyroid and my TSH was 0.15 T3 and T4 is normal . Also i have b12 deficiency and vitamin D deficiency so I'm taking weekly supplement of vitamin D and 1000 mcg b12 daily. I'm taking homeopathy medicine for 40 days and took 2nd test after 30 days. My TSH is 0.5 now. I feel a little bit relief from weakness and depression but I'm facing with 2 new problem from last week that is breathtaking problem and very rapid heartrate. I just want to know if i should start allopathy medicine or homeopathy is okay? Bcs i heard that thyroid take time to start recover. So please let me know if both of medicines take same time. Because some of my friends advising me to start allopathy and never take a chance as i can develop some serious problems.Sorry for my poor english😐Thank you.\n"],
 'summary': ['What are the treatments for hyperthyroidism?']}

In [0]:
light_result["summary"]

Out[27]: ['What are the treatments for hyperthyroidism?']

## 📃 summarizer_radiology

This model is capable of summarizing radiology reports while preserving the important information.

In [0]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
    
summarizer = medical.Summarizer()\
    .pretrained("summarizer_radiology", "en", "clinical/models")\
    .setInputCols(["document"])\
    .setOutputCol("summary")\
    .setMaxTextLength(512)\
    .setMaxNewTokens(512)

pipeline = nlp.Pipeline(stages=[
    document_assembler,
    summarizer  
])

model = pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))
light_model = nlp.LightPipeline(model)

summarizer_radiology download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]

In [0]:
text = """INDICATIONS: Peripheral vascular disease with claudication.

RIGHT:
1. Normal arterial imaging of right lower extremity.
2. Peak systolic velocity is normal.
3. Arterial waveform is triphasic.
4. Ankle brachial index is 0.96.

LEFT:
1. Normal arterial imaging of left lower extremity.
2. Peak systolic velocity is normal.
3. Arterial waveform is triphasic throughout except in posterior tibial artery where it is biphasic.
4. Ankle brachial index is 1.06.

IMPRESSION: 
Normal arterial imaging of both lower lobes.
"""

light_result = light_model.annotate(text)
light_result


Out[29]: {'document': ['INDICATIONS: Peripheral vascular disease with claudication.\n\nRIGHT:\n1. Normal arterial imaging of right lower extremity.\n2. Peak systolic velocity is normal.\n3. Arterial waveform is triphasic.\n4. Ankle brachial index is 0.96.\n\nLEFT:\n1. Normal arterial imaging of left lower extremity.\n2. Peak systolic velocity is normal.\n3. Arterial waveform is triphasic throughout except in posterior tibial artery where it is biphasic.\n4. Ankle brachial index is 1.06.\n\nIMPRESSION: \nNormal arterial imaging of both lower lobes.\n'],
 'summary': ['The patient has peripheral vascular disease with claudication. The right lower extremity shows normal arterial imaging, but the peak systolic velocity is normal. The arterial waveform is triphasic throughout, except for the posterior tibial artery, which is biphasic. The ankle brachial index is 0.96. The impression is normal arterial imaging of both lower lobes.']}

In [0]:
light_result["summary"]

Out[30]: ['The patient has peripheral vascular disease with claudication. The right lower extremity shows normal arterial imaging, but the peak systolic velocity is normal. The arterial waveform is triphasic throughout, except for the posterior tibial artery, which is biphasic. The ankle brachial index is 0.96. The impression is normal arterial imaging of both lower lobes.']

## 📃 summarizer_clinical_guidelines_large

This innovative Medical Summarizer Model is adept at providing succinct summarizations of clinical guidelines.

In [0]:
document = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

summarizer = medical.Summarizer.pretrained("summarizer_clinical_guidelines_large", "en", "clinical/models")\
    .setInputCols(["document"])\
    .setOutputCol("summary")\
    .setMaxTextLength(768)\
    .setMaxNewTokens(512)

pipeline = nlp.Pipeline(stages=[
    document,
    summarizer  
])

In [0]:
text = """Clinical Guidelines for Breast Cancer:
Breast cancer is the most common type of cancer among women. It occurs when the cells in the breast start growing abnormally, forming a lump or mass. This can result in the spread of cancerous cells to other parts of the body. Breast cancer may occur in both men and women but is more prevalent in women.
The exact cause of breast cancer is unknown. However, several risk factors can increase your likelihood of developing breast cancer, such as:
- A personal or family history of breast cancer
- A genetic mutation, such as BRCA1 or BRCA2
- Exposure to radiation
- Age (most commonly occurring in women over 50)
- Early onset of menstruation or late menopause
- Obesity
- Hormonal factors, such as taking hormone replacement therapy
Breast cancer may not present symptoms during its early stages. Symptoms typically manifest as the disease progresses. Some notable symptoms include:
- A lump or thickening in the breast or underarm area
- Changes in the size or shape of the breast
- Nipple discharge
- Nipple changes in appearance, such as inversion or flattening
- Redness or swelling in the breast
Treatment for breast cancer depends on several factors, including the stage of the cancer, the location of the tumor, and the individual's overall health. Common treatment options include:
- Surgery (such as lumpectomy or mastectomy)
- Radiation therapy
- Chemotherapy
- Hormone therapy
- Targeted therapy
Early detection is crucial for the successful treatment of breast cancer. Women are advised to routinely perform self-examinations and undergo regular mammogram testing starting at age 40. If you notice any changes in your breast tissue, consult with your healthcare provider immediately.""" 
 
data = spark.createDataFrame([[text]]).toDF("text")
data.show(truncate = 60)

In [0]:
model = pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

result = model.transform(data)
result.show()

In [0]:
result.select("summary.result").show(truncate=False)

# Medical Text Summarization Comparison

## Spark NLP for Healthcare vs Other SOTA Models

**Flan-T5-base-samsum**

- model_name = "philschmid/flan-t5-base-samsum"
- model_size = 250M
- base_model = flan-t5
- dataset = samsum
- domain = general
- owner = google (fine-tuned)
- code_availibilty = fine tunning code is not available
- checkpoints_availaibility = Available
- link_to_repo = https://huggingface.co/philschmid/flan-t5-base-samsum/tree/main

Reported metrics
- Loss: 1.3716
- Rouge1: 47.2358
- Rouge2: 23.5135
- Rougel: 39.6266
- Rougelsum: 43.3458
- Gen Len: 17.3907

**Flan-T5-base**

- model_name = "google/flan-t5-base"
- model_size = 250M
- base_model = flan-t5
- domain = general
- owner = google
- checkpoints_availaibility = Available
- link_to_repo = https://huggingface.co/google/flan-t5-base

**Pegasus Samsum**

- model_name = transformersbook/pegasus-samsum
- model_size = 570M
- base_model = google/pegasus-cnn_dailymail
- dataset = samsum
- domain = general
- owner = google (fine-tunned)
- code_availibilty = https://github.com/nlp-with-transformers/notebooks/blob/main/06_summarization.ipynb
- checkpoints_availaibility = Available
- link_to_repo = https://github.com/nlp-with-transformers/notebooks/blob/main/06_summarization.ipynb

**Bart-large-samsum**

- model_name = linydub/bart-large-samsum
- model_size = 500M
- base_model = facebook/bart-large
- dataset = samsum
- domain = general
- owner = facebook (fine-tuned)
- code_availibilty = fine tunning code is not available
- checkpoints_availaibility = https://huggingface.co/linydub/bart-large-samsum
- link_to_repo = https://github.com/linydub/azureml-greenai-txtsum

Reported metrics
- eval_rouge1	55.0234
- eval_rouge2	29.6005
- eval_rougeL	44.914
- eval_rougeLsum	50.464
- predict_rouge1	53.4345
- predict_rouge2	28.7445
- predict_rougeL	44.1848
- predict_rougeLsum	49.1874

**Bart-large-cnn-samsum**

- model_name = philschmid/bart-large-cnn-samsum
- model_size = 500M
- base_model = facebook/bart-large
- dataset = samsum
- domain = general
- owner = facebook (fine-tuned)
- code_availibilty = fine tunning code is not available
- checkpoints_availaibility = Available
- link_to_repo = https://huggingface.co/philschmid/bart-large-cnn-samsum

## Comparison

In [0]:
# df[['original','flan-t5-base-samsum', 'flan-t5-base', 'bart-large-samsum','pegasus-samsum', 
#     'pegasus-xsum', 'bart-large-cnn-samsum', 'sparknlp_aug', 'sparknlp_slim']].T.to_markdown()

|                       | radiology_text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | clinical_text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
|-----------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| original              | INDICATIONS: Peripheral vascular disease with claudication.\n\nRIGHT:\n1. Normal arterial imaging of right lower extremity.\n2. Peak systolic velocity is normal.\n3. Arterial waveform is triphasic.\n4. Ankle brachial index is 0.96.\n\nLEFT:\n1. Normal arterial imaging of left lower extremity.\n\n2. Peak systolic velocity is normal.\n3. Arterial waveform is triphasic throughout except in posterior tibial artery where it is biphasic.\n4. Ankle brachial index is 1.06.\n\nIMPRESSION:\nNormal arterial imaging of both lower extremities.\n | Patient with hypertension, syncope, and spinal stenosis - for recheck.  (Medical Transcription Sample Report) SUBJECTIVE:  The patient is a 78-year-old female who returns for recheck. She has hypertension. She denies difficulty with chest pain, palpations, orthopnea, nocturnal dyspnea, or edema. PAST MEDICAL HISTORY / SURGERY / HOSPITALIZATIONS: Reviewed and unchanged from the dictation on 12/03/2003. MEDICATIONS:  Atenolol 50 mg daily, Premarin 0.625 mg daily, calcium with vitamin D two to three pills daily, multivitamin daily, aspirin as needed, and TriViFlor 25 mg two pills daily. She also has Elocon cream 0.1% and Synalar cream 0.01% that she uses as needed for rash. |
| flan-t5-base-samsum   | RIGHT: 1. Normal arterial imaging of right lower extremity. 2. Peak systolic velocity is normal. 3. Arterial waveform is triphasic throughout except in posterior tibial artery where it is biphasic. 4. Ankle brachial index is 1.06. IMPRESSION: Normal arterial imaging of both lower extremities.                                                                                                                                                                                                                                                      | A patient with hypertension, syncope, and spinal stenosis for recheck.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| flan-t5-base          | RIGHT: 1. Normal arterial imaging of right lower extremity. 2. Peak systolic velocity is normal. 3. Arterial waveform is triphasic throughout except in posterior tibial artery where it is biphasic. 4. Ankle brachial index is 0.06                                                                                                                                                                                                                                                                                                                      | - recheck of a patient with hypertension, syncope, and spinal stenosis                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| bart-large-samsum     | Peripheral vascular disease with claudication. Arterial waveform is triphasic. Peak systolic velocity is normal. Ankle brachial index is 1.06.                                                                                                                                                                                                                                                                                                                                                                                                             | A 78-year-old female with hypertension, syncope, and spinal stenosis returns for recheck. She has Atenolol 50 mg daily, Premarin 0.625 mg daily Premarin, calcium with vitamin D two to three pills daily, multivitamin daily                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
| pegasus-samsum        | Arterial waveform is triphasic throughout except in posterior tibial artery where it is biphasic. IMPRESSION: Normal arterial imaging of both lower extremities.                                                                                                                                                                                                                                                                                                                                                                                           | The patient is a 78-year-old female who returns for recheck. She has hypertension. She denies difficulty with chest pain, palpations, orthopnea, nocturnal dyspnea, or edema. She has Atenolol 50 mg daily, Premarin 0.625 mg daily, calcium with vitamin D two to three pills daily, multivitamin daily, aspirin as needed, and TriViFlor 25 mg two pills daily.                                                                                                                                                                                                                                                                                                                                       |
| pegasus-xsum          | Arterial imaging of both lower extremities.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | A case report of a 78-year-old woman with hypertension, syncope, and spinal stenosis.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
| bart-large-cnn-samsum | Peripheral vascular disease with claudication. Arterial waveform is triphasic throughout except in posterior tibial artery where it is biphasic. Ankle brachial index is 1.06.    IMAGINATION: Normal arterial imaging of both lower extremities.                                                                                                                                                                                                                                                                                                          | The patient is 78-year-old female with hypertension, syncope, and spinal stenosis. She has Atenolol 50 mg daily, Premarin 0.625 mg daily and calcium with vitamin D two to three pills daily, multivitamin daily, aspirin as needed and TriViFlor 25 mg two pills daily. She also has Elocon cream 0.1% and Synalar cream  0.01%.                                                                                                                                                                                                                                                                                                                                                                       |
| sparknlp_aug          | The patient has peripheral vascular disease with claudication and underwent normal arterial imaging of both lower extremities. The right lower extremity showed normal arterial imaging with normal peak systolic velocity, triphasic arterial waveform, and ankle brachial index of 0.96. The left lower extremity showed normal arterial imaging with triphasic arterial waveform except for the posterior tibial artery where it was biphasic. The ankle brachial index was 0.06.                                                                       | A 78-year-old female with hypertension, syncope, and spinal stenosis returns for a recheck. She denies difficulty with chest pain, palpations, orthopnea, nocturnal dyspnea, or edema. Her medications include Atenolol, Premarin, calcium with vitamin D, multivitamin, aspirin, and TriViFlor. She also has Elocon cream and Synalar cream for rash.                                                                                                                                                                                                                                                                                                                                                  |
| sparknlp_slim         | The patient has peripheral vascular disease with claudication and underwent normal arterial imaging of both lower extremities. The peak systolic velocity is normal, but the arterial waveform is triphasic throughout, except for the posterior tibial artery where it is biphasic. The ankle brachial index is 0.06. The impression is that the arterial imaging of both lower extremities is normal.                                                                                                                                                    | A 78-year-old female with hypertension, syncope, and spinal stenosis returns for recheck. She denies chest pain, palpations, orthopnea, nocturnal dyspnea, or edema. She is on multiple medications and has Elocon cream and Synalar cream for rash.                                                                                                                                                                                                                                                                                                                                                                                                                                                    |

### Summarization with GPT-4

In [0]:
import nltk
from bert_score import score

def generate_scores(res, ref):
    
    berts_p, berts_r, berts_f = score([res], [ref], lang="en", return_hash=False)
    berts_p, berts_r, berts_f = round(float(berts_p[0]), 4), round(float(berts_r[0]), 4), round(float(berts_f[0]), 4)

    print('BERT Score Precision:', berts_p)
    print('BERT Score Recall:', berts_r)
    print('BERT Score F1:', berts_f)


In [0]:
radiology_summary_gpt4 = '''The radiology report indicates that the patient has peripheral vascular disease with claudication. The arterial imaging of both lower extremities is normal. The peak systolic velocity is normal, and the arterial waveform is triphasic in both extremities except for the posterior tibial artery in the left extremity, which is biphasic. The ankle brachial index values are 0.96 and 1.06 for the right and left extremities, respectively.'''

radiology_summary_gpt4


Out[42]: 'The radiology report indicates that the patient has peripheral vascular disease with claudication. The arterial imaging of both lower extremities is normal. The peak systolic velocity is normal, and the arterial waveform is triphasic in both extremities except for the posterior tibial artery in the left extremity, which is biphasic. The ankle brachial index values are 0.96 and 1.06 for the right and left extremities, respectively.'

In [0]:
# generate_scores(summary_dict['sparknlp_slim']['radiology_text'], radiology_summary_gpt4)


# BERT Score Precision: 0.9614
# BERT Score Recall: 0.9453
# BERT Score F1: 0.9533

In [0]:
# generate_scores(summary_dict['sparknlp_aug']['radiology_text'], radiology_summary_gpt4)
# 
# BERT Score Precision: 0.9358
# BERT Score Recall: 0.9285
# BERT Score F1: 0.9321

In [0]:
# generate_scores(summary_dict['bart-large-cnn-samsum']['radiology_text'], radiology_summary_gpt4)
# 
# BERT Score Precision: 0.9124
# BERT Score Recall: 0.9088
# BERT Score F1: 0.9106

In [0]:
# generate_scores(summary_dict['bart-large-cnn-samsum']['radiology_text'], radiology_summary_gpt4)
# 
# BERT Score Precision: 0.9124
# BERT Score Recall: 0.9088
# BERT Score F1: 0.9106

In [0]:
clinical_summary_gpt4 = '''The report is about a 78-year-old female patient with hypertension, syncope, and spinal stenosis who returns for a recheck. She denies experiencing chest pain, palpitations, orthopnea, nocturnal dyspnea, or edema. Her past medical history remains unchanged since the last dictation on 12/03/2003. The patient's medications include Atenolol, Premarin, calcium with vitamin D, multivitamin, aspirin, TriViFlor, Elocon cream, and Synalar cream.'''
clinical_summary_gpt4

Out[5]: "The report is about a 78-year-old female patient with hypertension, syncope, and spinal stenosis who returns for a recheck. She denies experiencing chest pain, palpitations, orthopnea, nocturnal dyspnea, or edema. Her past medical history remains unchanged since the last dictation on 12/03/2003. The patient's medications include Atenolol, Premarin, calcium with vitamin D, multivitamin, aspirin, TriViFlor, Elocon cream, and Synalar cream."

In [0]:
# generate_scores(summary_dict['sparknlp_slim']['clinical_text'], clinical_summary_gpt4)
# 
# BERT Score Precision: 0.9549
# BERT Score Recall: 0.8891
# BERT Score F1: 0.9208

In [0]:
# generate_scores(summary_dict['sparknlp_aug']['clinical_text'], clinical_summary_gpt4)
# 
# BERT Score Precision: 0.9597
# BERT Score Recall: 0.9311
# BERT Score F1: 0.9452

In [0]:
# generate_scores(summary_dict['bart-large-cnn-samsum']['clinical_text'], clinical_summary_gpt4)
# 
# BERT Score Precision: 0.8855
# BERT Score Recall: 0.8763
# BERT Score F1: 0.8809